In [1]:
import pandas as pd 

In [2]:
itinierarios_bases=pd.read_parquet('../Data/Itinerarios/itinerarios_bases3.parquet')
itinierarios_bases.head()

,user_type,travel_time,idunplug_station,ageRange,idplug_station,unplug_hourTime,status,Origen_destino,Número_Salida,Distrito_Salida,...,Barrio_Llegada,Número de Plazas_Llegada,Latitud_Llegada,Longitud_Llegada,name_Salida,address_Salida,name_Llegada,address_Llegada,dayofweek,hour
0,1,154,44,5,40,2019-07-01 00:00:00+00:00,short_rental,44-40,44,01 CENTRO,...,01-02 EMBAJADORES,24.0,40.413788,-3.704746,Plaza de Nelson Mandela,Calle Mesón de Paredes nº 35,Conde de Romanones,Plaza del Conde de Romanones nº 9,0,0
1,1,184,57,4,129,2019-07-01 00:00:00+00:00,short_rental,57-129,57,01 CENTRO,...,07-05 RÍOS ROSAS,24.0,40.438469,-3.698282,Santa Cruz del Marcenado,Calle Santa Cruz del Marcenado nº 24,José Abascal,Calle José Abascal nº 33,0,0
2,1,282,53,4,43,2019-07-01 00:00:00+00:00,short_rental,53-43,53,01 CENTRO,...,01-02 EMBAJADORES,24.0,40.410088,-3.702545,Plaza de Lavapiés,Calle Valencia nº 1,Jesús y María,Calle Lavapiés nº 34-36,0,0
3,1,388,56,0,13,2019-07-01 00:00:00+00:00,short_rental,56-13,56,01 CENTRO,...,01-05 UNIVERSIDAD,24.0,40.427364,-3.710473,Plaza del Carmen,Plaza del Carmen nº 1,Conde Duque,Calle Conde Duque nº 22,0,0
4,1,254,105,0,75,2019-07-01 00:00:00+00:00,short_rental,105-75,105,04 SALAMANCA,...,03-05 JERÓNIMOS,24.0,40.408340,-3.678415,Serrano,Calle Serrano nº 54,Menéndez Pelayo,Avenida de Menéndez Pelayo nº 90,0,0


In [4]:
itinierarios_bases.columns

Index(['user_type', 'travel_time', 'idunplug_station', 'ageRange',
       'idplug_station', 'unplug_hourTime', 'status', 'Origen_destino',
       'Número_Salida', 'Distrito_Salida', 'Barrio_Salida',
       'Número de Plazas_Salida', 'Latitud_Salida', 'Longitud_Salida',
       'Número_Llegada', 'Distrito_Llegada', 'Barrio_Llegada',
       'Número de Plazas_Llegada', 'Latitud_Llegada', 'Longitud_Llegada',
       'name_Salida', 'address_Salida', 'name_Llegada', 'address_Llegada',
       'dayofweek', 'hour'],
      dtype='object')

In [35]:
def DistrubicionEstaciones(itinerarios_bases): 
    cols_grup=['idplug_station', 'dayofweek', 'hour']
    df_llegada = itinierarios_bases.groupby(cols_grup)['status'].count().to_frame().reset_index()
    df_llegada.rename(columns={'status': 'viajes_llegada', 'idplug_station': 'estacion'}, inplace=True)
    # Percentage by lambda and DataFrame.apply() method.

    df3 = df_llegada.groupby(['dayofweek', 'hour'])['viajes_llegada'].sum().to_dict()#.reset_index()
    df_llegada['dict']=df_llegada.apply(lambda x: (x.dayofweek,x.hour),axis=1)
    df_llegada['viajes_llegada%']=df_llegada.apply(lambda x: x.viajes_llegada/df3[x.dict], axis=1)
    df_llegada_def=df_llegada[['estacion','dayofweek', 'hour', 'viajes_llegada', 'viajes_llegada%' ]]
    

    cols_grup2=['idunplug_station', 'dayofweek', 'hour']
    df_salida = itinierarios_bases.groupby(cols_grup2)['status'].count().to_frame().reset_index()
    df_salida.rename(columns={'status': 'viajes_salida', 'idunplug_station': 'estacion'}, inplace=True)
    # Percentage by lambda and DataFrame.apply() method.

    df3 = df_salida.groupby(['dayofweek', 'hour'])['viajes_salida'].sum().to_dict()#.reset_index()
    df_salida['dict']=df_salida.apply(lambda x: (x.dayofweek,x.hour),axis=1)
    df_salida['viajes_salida%']=df_salida.apply(lambda x: x.viajes_salida/df3[x.dict], axis=1)
    df_salida_def=df_salida[['estacion','dayofweek', 'hour', 'viajes_salida', 'viajes_salida%' ]]
    

    return pd.merge(df_llegada_def,df_salida_def, how='inner', on=['estacion','dayofweek', 'hour'])

   estacion  dayofweek  hour  viajes_llegada  viajes_llegada%
0         1          0     0               8         0.002851
1         1          0     1               7         0.003989
2         1          0     2              10         0.008354
3         1          0     3              20         0.018832
4         1          0     4              30         0.013743
   estacion  dayofweek  hour  viajes_salida  viajes_salida%
0         1          0     0             33        0.011761
1         1          0     1             20        0.011396
2         1          0     2             17        0.014202
3         1          0     3             11        0.010358
4         1          0     4              9        0.004123


,estacion,dayofweek,hour,viajes_llegada,viajes_llegada%,viajes_salida,viajes_salida%
0,1,0,0,8,0.002851,33,0.011761
1,1,0,1,7,0.003989,20,0.011396
2,1,0,2,10,0.008354,17,0.014202
3,1,0,3,20,0.018832,11,0.010358
4,1,0,4,30,0.013743,9,0.004123
...,...,...,...,...,...,...,...
34368,219,6,19,42,0.004241,47,0.004746
34369,219,6,20,22,0.002779,31,0.003916
34370,219,6,21,27,0.003796,33,0.004640
34371,219,6,22,17,0.002789,35,0.005742
